# Simulating Anything - Demo Notebook

This notebook demonstrates the full Simulating Anything pipeline:
natural language research question → simulation → analysis → discovery report.

We show three examples, one for each V1 domain:
1. **Reaction-Diffusion**: Gray-Scott pattern formation
2. **Rigid-Body**: Projectile motion with drag
3. **Agent-Based**: Lotka-Volterra predator-prey dynamics

In [ ]:
import logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s - %(name)s - %(message)s')

In [ ]:
from simulating_anything import Pipeline
print(f'Simulating Anything loaded successfully')

## Example 1: Reaction-Diffusion (Gray-Scott)

Investigating how feed and kill rates affect pattern formation.

In [ ]:
# Running the simulation directly (without LLM agents)
from simulating_anything.simulation.reaction_diffusion import GrayScottSimulation
from simulating_anything.types.simulation import SimulationConfig, Domain, SimulationBackend

config = SimulationConfig(
    domain=Domain.REACTION_DIFFUSION,
    backend=SimulationBackend.JAX_FD,
    grid_resolution=(64, 64),
    domain_size=(2.5, 2.5),
    dt=1.0,
    n_steps=500,
    parameters={'D_u': 0.16, 'D_v': 0.08, 'f': 0.035, 'k': 0.065},
)

sim = GrayScottSimulation(config)
traj = sim.run()
print(f'States shape: {traj.states.shape}')
print(f'Steps: {traj.n_steps}')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, axes = plt.subplots(1, 3, figsize=(15, 4))
for i, step in enumerate([0, 250, 500]):
    axes[i].imshow(traj.states[step, :, :, 1], cmap='viridis')
    axes[i].set_title(f'v-field at t={step}')
    axes[i].axis('off')
plt.suptitle('Gray-Scott Pattern Formation')
plt.tight_layout()
plt.show()

## Example 2: Rigid-Body (Projectile with Drag)

Studying how air drag affects projectile range.

In [ ]:
from simulating_anything.simulation.rigid_body import ProjectileSimulation

config = SimulationConfig(
    domain=Domain.RIGID_BODY,
    grid_resolution=(1,),
    domain_size=(100.0, 100.0),
    dt=0.01,
    n_steps=2000,
    parameters={
        'gravity': 9.81, 'drag_coefficient': 0.1,
        'mass': 1.0, 'initial_speed': 30.0, 'launch_angle': 45.0
    },
)

sim = ProjectileSimulation(config)
traj = sim.run()
print(f'States shape: {traj.states.shape}')

# Plot trajectory
x = traj.states[:, 0]
y = traj.states[:, 1]
mask = y >= 0

plt.figure(figsize=(10, 5))
plt.plot(x[mask], y[mask], 'b-', linewidth=2)
plt.xlabel('Distance (m)')
plt.ylabel('Height (m)')
plt.title('Projectile with Quadratic Drag')
plt.grid(True, alpha=0.3)
plt.show()
print(f'Range: {x[mask][-1]:.2f} m')

## Example 3: Agent-Based (Lotka-Volterra)

Predator-prey population dynamics.

In [ ]:
from simulating_anything.simulation.agent_based import LotkaVolterraSimulation

config = SimulationConfig(
    domain=Domain.AGENT_BASED,
    backend=SimulationBackend.DIFFRAX,
    grid_resolution=(1,),
    domain_size=(1.0,),
    dt=0.01,
    n_steps=5000,
    parameters={
        'alpha': 1.1, 'beta': 0.4, 'gamma': 0.4, 'delta': 0.1,
        'prey_0': 40.0, 'predator_0': 9.0
    },
)

sim = LotkaVolterraSimulation(config)
traj = sim.run()
print(f'States shape: {traj.states.shape}')

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Time series
ax1.plot(traj.timestamps, traj.states[:, 0], 'g-', label='Prey')
ax1.plot(traj.timestamps, traj.states[:, 1], 'r-', label='Predator')
ax1.set_xlabel('Time')
ax1.set_ylabel('Population')
ax1.set_title('Population Dynamics')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Phase portrait
ax2.plot(traj.states[:, 0], traj.states[:, 1], 'b-', alpha=0.7)
ax2.plot(traj.states[0, 0], traj.states[0, 1], 'go', markersize=10, label='Start')
ax2.set_xlabel('Prey')
ax2.set_ylabel('Predator')
ax2.set_title('Phase Portrait')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Verification Checks

Running conservation and positivity checks on the simulations.

In [ ]:
from simulating_anything.verification.conservation import (
    check_mass_conservation, check_positivity
)

# Check Lotka-Volterra positivity
pos_check = check_positivity(traj.states, 'populations')
print(f'Positivity check: {"PASSED" if pos_check.passed else "FAILED"}')
print(f'  {pos_check.message}')

## Full Pipeline (requires Claude Code CLI)

Uncomment and run the cell below to execute the full pipeline.
This requires the `claude` CLI to be installed and authenticated.

In [ ]:
# Uncomment to run full pipeline:
# pipeline = Pipeline(output_dir='output/demo')
# report = pipeline.run(
#     'How do feed rate and kill rate affect pattern formation '
#     'in Gray-Scott reaction-diffusion systems?'
# )
# print(report)